In [1]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

## HP Search Toxic Datasets

In [2]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    if "alpha" in config.keys():
        config_list.append('+ue.alpha\={}'.format(config['alpha']))
        config_list.append('+ue.coverage\={}'.format(config['coverage']))

    config_list.append('ue.calibrate\=False')
    config_list.append('training\=electra_base')
    
    return config_list

In [3]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    j = 0
    n_devices = len(cuda_devices)
    for i, mc_configs in enumerate(configs):
        for conf in expand_config(mc_configs):
            for task_name in tasks:

                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG_PATH=../configs/{task_name}.yaml python ./run_glue_with_hyp_search.py'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                coverage = ''
                if "coverage" in conf.keys():
                    coverage = '_{}'.format(conf['coverage'])
                
                if 'model_name_or_path' in conf.keys():
                    model = conf['model_name_or_path']
                else:
                    model = 'google/electra-base-discriminator'   
                
                model_name = model.split('/')[-1].split('-')[0]
                new_task += f' {args}'
                new_task += f' hydra.run.dir=../workdir/hp_search/{model_name}_{reg}_{use_spectralnorm}{coverage}/{task_name}'
                
                new_task += f" model.model_name_or_path='{model}'"
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [4]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

train_configs = {
    'reg_type': ['raw'],
    'use_spectralnorm': ['True'],
}
    
cuda_devices = [0, 1, 2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'electra_hp_search_hue_datasets.sh')

In [5]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

train_configs = {
    'reg_type': ['raw'],
    'use_spectralnorm': ['True'],
    'model_name_or_path': ['bert-base-uncased']
}
    
cuda_devices = [0, 1, 2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'bert_hp_search_hue_datasets.sh')

In [6]:
tasks = ['paradetox', 'toxigen', '20newsgroups']

train_configs = {
    'reg_type': ['selectivenet'],
    'use_spectralnorm': ['False'],# 'True'],
    'alpha': [0.5],
    'coverage': [0.5, 0.7, 0.8, 0.85, 0.9, 0.95]
}
    
cuda_devices = [0, 1, 2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'electra_hp_search_hue_datasets_selectivenet.sh')

## Train/Validation for fitting HP for Hybrid UE

In [7]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ue\=mc')
    config_list.append('do_ue_estimate\=False')
    config_list.append('+ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    if 'model_name_or_path' in config.keys():
        config_list.append('model.model_name_or_path\={}'.format(config['model_name_or_path']))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(config['eval_subsample']))
    
    config_list.append('training\=electra_base')
    return config_list

In [8]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices} seeds=[23419,705525,4837,10671619,1084218,43]'
                new_task += f' script=run_glue_method_hp.py'
                
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                if 'model_name_or_path' in conf.keys():
                    model = conf['model_name_or_path']
                else:
                    model = 'google/electra-base-discriminator'     
                model_name = model.split('/')[-1].split('-')[0]

                try:
                    with open(f'../../optimal_hyperparameters/{model_name}_{reg}_{use_spectralnorm}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)
                except:
                    print(f'not exists {model_name}_{reg}_{use_spectralnorm}/{task_name}')
                    continue

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                lamb_intra = params['lamb_intra']
                lamb = params['lamb']
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                eval_subsample = conf['eval_subsample']
                new_task+=f' output_dir=../workdir/run_train_models_method_hp/{model_name}_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [9]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0],
    'eval_subsample': [0.2],
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 
              'run_train_electra_models_toxic_method_hp.sh')

In [10]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0],
    'eval_subsample': [0.2],
    'model_name_or_path': ['bert-base-uncased']
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 
              'run_train_bert_models_toxic_method_hp.sh')

In [11]:
import os 

def generate_eval_ue_str_config(config):
    config_list = []
    config_list.append('ue\={}'.format(config['ue']))
    config_list.append('do_ue_estimate\=True')
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    
    config_list.append('ue.calibrate\={}'.format(config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(config['eval_subsample']))
    
    if config['ue'] == 'nuq':
        config_list.append('ue.nuq.n_neighbors\=50')
        config_list.append('ue.nuq.log_pN\=0')
        config_list.append('+ue.nuq.n_points\=50')
        config_list.append('+ue.nuq.n_folds\=20')
        config_list.append('+ue.nuq.n_samples\=5')
    
    config_list.append('+ue.use_tanh\={}'.format(config['use_tanh']))
    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash_ue(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                new_task += f' script=run_glue_method_hp.py'
                
                eval_subsample = conf['eval_subsample']
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                if 'model_name_or_path' in conf.keys():
                    model = conf['model_name_or_path']
                else:
                    model = 'google/electra-base-discriminator'     
                model_name = model.split('/')[-1].split('-')[0]
                
            
                use_tanh = '_tanh' if conf['use_tanh'] == 'True' else ''
                base_path=f' output_dir=../workdir/run_tasks_for_model_series_method_hp_64{use_tanh}/{model_name}_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}'

                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models_method_hp/{model_name}_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/models/{task_name}'
                #model_series_dir = f'~/hue_workdir_eacl/run_train_models_method_hp/{model_name}_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/models/{task_name}'
                
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [12]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

ue_configs = {
    'ue': ['mahalanobis', 'rde', 'ddu'],
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0],
    'eval_subsample': [0.2],
    'use_tanh': ['False'],
}
    
cuda_devices = '[0]'

generate_bash_ue([ue_configs], cuda_devices, tasks, [generate_eval_ue_str_config], 
                 file_name='run_eval_electra_models_toxic_method_hp.sh')

In [13]:
tasks = ['paradetox', 'toxigen','implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

ue_configs = {
    'ue': ['mahalanobis', 'rde', 'ddu'],
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0],
    'eval_subsample': [0.2],
    'model_name_or_path': ['bert-base-uncased'],
    'use_tanh': ['False'],
}
    
cuda_devices = '[0]'

generate_bash_ue([ue_configs], cuda_devices, tasks, [generate_eval_ue_str_config], 
                 file_name='run_eval_bert_models_toxic_method_hp.sh')

## Final Exps Toxic Datasets

In [14]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    if 'model_name_or_path' in dpp_config.keys():
        config_list.append('model.model_name_or_path\={}'.format(dpp_config['model_name_or_path']))
        
    if "alpha" in dpp_config.keys():
        config_list.append('+ue.alpha\={}'.format(dpp_config['alpha']))
        config_list.append('+ue.coverage\={}'.format(dpp_config['coverage']))

    config_list.append('++ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('ue.calibrate\=False')
    config_list.append('training\=electra_base')
    
    return config_list

In [15]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices} seeds=[23419,705525,4837,10671619,1084218,43]'
                new_task += f' script=run_glue.py'
                args = ' '.join(generate_func[i](conf))

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'

                if 'model_name_or_path' in conf.keys():
                    model = conf['model_name_or_path']
                else:
                    model = 'google/electra-base-discriminator'     
                model_name = model.split('/')[-1].split('-')[0]
                
                coverage = ""
                if "coverage" in conf.keys():
                    coverage = '_{}'.format(conf['coverage'])

                with open(f'../../optimal_hyperparameters/{model_name}_{reg}_{use_spectralnorm}{coverage}/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'

                try:
                    margin = params['margin']
                    gamma_pos = params['gamma_pos']
                    gamma_neg = params['gamma_neg']
                except:
                    margin = 0
                    gamma_pos = 1
                    gamma_neg = 4
                    
                args += f' +ue.margin\={margin}'
                args += f' +ue.gamma_pos\={gamma_pos}'
                args += f' +ue.gamma_neg\={gamma_neg}'

                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'

                new_task+=f' output_dir=../workdir/run_train_models/{model_name}_{reg}_{use_spectralnorm}_opt{coverage}/{task_name}'

                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [16]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

train_configs = {
    'reg_type': ['raw'],
    'use_spectralnorm': ['True'],
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_electra_toxic.sh')

In [17]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

train_configs = {
    'reg_type': ['raw'],
    'use_spectralnorm': ['True'],
    'model_name_or_path': ['bert-base-uncased']
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_bert_toxic.sh')

In [18]:
tasks = ['paradetox', 'toxigen', '20newsgroups']

train_configs = {
    'reg_type': ['selectivenet'],
    'use_spectralnorm': ['False'],
    'alpha': [0.5],
    'coverage': [0.5, 0.7, 0.8, 0.85, 0.9, 0.95]
}
    
cuda_devices = '[0,1,2]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_electra_selectivenet_toxic.sh')

# Deep Ensemble

In [19]:
import yaml
from pathlib import Path

def generate_bash(cuda_devices, tasks, models, filename="../../scripts/miscl_scripts/run_train_ensemble_models_toxic.sh"):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series.yaml python ./run_train_ensemble_series.py'
    for model in models:
        for task_name in tasks:
            task_configs = f'{task_name}.yaml'
            conf = yaml.safe_load(Path(f'../../configs/{task_configs}').read_text())
            subsample_perc = conf['data']['subsample_perc']
            if subsample_perc == 0:
                subsample_perc = 0.9
            else:
                subsample_perc *= 0.9
                
            subsample_perc = round(subsample_perc, 2)
            args = f'ue.calibrate\=False data.subsample_perc\={subsample_perc} +ue.use_spectralnorm\=True'
            
            if model == 'bert':
                model_type = ' model.model_name_or_path\=bert-base-uncased'
                args += model_type
            if model == 'deberta':
                model_type = ' model.model_name_or_path\=microsoft/deberta-v3-base'
                args += model_type
            if model == 'distilbert':
                model_type = ' model.model_name_or_path\=distilbert-base-uncased'
                args += model_type
            if model == 'distilroberta':
                model_type = ' model.model_name_or_path\=distilroberta-base'
                args += model_type

            use_spectralnorm_params = ''
            reg_params = 'raw' 

            with open(f'../../optimal_hyperparameters/{model}_raw_sn/{task_name}/optimal_hyperparameters.yaml', 'r') as f:
                params = yaml.load(f, Loader=yaml.FullLoader)

            num_train_epochs = params['num_train_epochs']
            args += f' training.num_train_epochs\={num_train_epochs}'
            learning_rate = params['learning_rate']
            args += f' training.learning_rate\={learning_rate}'
            per_device_train_batch_size = params['per_device_train_batch_size']
            args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
            weight_decay = params['weight_decay']
            args += f' +training.weight_decay\={weight_decay}'       

            output_dir = f'../workdir/run_train_ensemble_series_90/{task_name}/{model}_raw_sn/'

            new_task = copy.deepcopy(base_arg)
            new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
            new_task += f' script=run_glue.py'
            full_config += '\n' + new_task if len(full_config) else new_task
    with open (filename, 'w') as rsh:
        rsh.write(full_config)

In [20]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', '20newsgroups', 'amazon']
models = ['electra', 'bert']
cuda_devices = '[0,1,2]'

generate_bash(cuda_devices, tasks, models)

In [21]:
import yaml
from pathlib import Path

def generate_bash_eval_ens(cuda_devices, tasks, models, time='', file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_ensemble_series.yaml python run_tasks_for_ensemble_series.py'
    for model in models:
        for task_name in tasks:
            task_configs = f'{task_name}.yaml'

            new_task = copy.deepcopy(base_arg)
            new_task += f' cuda_devices={cuda_devices} args=\'ue\=mc +ue.use_spectralnorm\=True\''
            new_task += f' config_path=../configs/{task_configs}'

            ensemble_series_dir = f'../workdir/run_train_ensemble_series_90/{task_name}/{model}_raw_sn/ensembles/'
            new_task+=f' ensemble_series_dir={ensemble_series_dir}'
        
            output_dir = f'../workdir/run_tasks_for_ensemble_series_90/{task_name}/{model}_raw_sn/ensembles{time}'
            new_task+=f' output_dir={output_dir}'
            new_task += f' script=run_glue.py'

            calc_output_dir = f'../workdir/run_calc_ues_metrics_90/{model}_raw_sn/{task_name}/ensemble'
            calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir={output_dir}/final_results extract_config=False output_dir={calc_output_dir};'

            full_config += '\n' + new_task if len(full_config) else new_task
            full_config += '\n' + calc_command
            
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [22]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', '20newsgroups', 'amazon']
models = ['electra', 'bert']
cuda_devices = '[0,1,2]'

generate_bash_eval_ens(cuda_devices, tasks, models, file_name='run_eval_ensemble_series_toxic.sh')

## Eval new estimators

In [23]:
def generate_ue_eval_methods_str_config(config):
    config_list = []
    
    config_list.append('ue\={}'.format(config['ue']))

    use_selective = 'False' if config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(config['reg_type']))
    config_list.append('do_ue_estimate\=True')
        
    if config['epistemic_ue'] == 'mahalanobis':
        config_list.append('ue.epistemic_key\=mahalanobis_distance')
        config_list.append('ue.epistemic.ue_type\=maha')
        
    if config['epistemic_ue'] == 'rde':
        config_list.append('ue.epistemic_key\=mahalanobis_distance')
        config_list.append('ue.epistemic.ue_type\=rde')
        config_list.append('+ue.epistemic.use_pca\=True')
        config_list.append('+ue.epistemic.use_mcd\=True')
        config_list.append('+ue.epistemic.use_class_cov\=True')
        config_list.append('+ue.epistemic.select_predict_dists\=True')
                
    if config['epistemic_ue'] == 'ddu':
        config_list.append('ue.epistemic_key\=ddu_scores')
        config_list.append('ue.epistemic.ue_type\=ddu')
        
    if config['epistemic_ue'] == 'nuq':
        config_list.append('ue.epistemic_key\=epistemic')
        config_list.append('ue.epistemic.ue_type\=nuq')
        config_list.append('+ue.epistemic.nuq.n_neighbors\=50')
        config_list.append('+ue.epistemic.nuq.log_pN\=0')
        config_list.append('+ue.epistemic.nuq.n_points\=50')
        config_list.append('+ue.epistemic.nuq.n_folds\=20')
        config_list.append('+ue.epistemic.nuq.n_samples\=5')
        config_list.append('+ue.epistemic.nuq.tune_bandwidth\=classification')
         
    if "aleatoric_method" in config.keys():
        config_list.append('ue.aleatoric\={}'.format(config['aleatoric_method']))
    config_list.append('++ue.use_spectralnorm\={}'.format(config['use_spectralnorm']))
    config_list.append('+ue.use_tanh\={}'.format(config['use_tanh']))
    if "alpha" in config.keys():
        config_list.append('+ue.alpha\={}'.format(config['alpha']))
        config_list.append('+ue.coverage\={}'.format(config['coverage']))
    
    reg = 'reg' if config['reg_type'] == 'reg-curr' else config['reg_type']
    if 'model_name_or_path' in config.keys():
        config_list.append('model.model_name_or_path\={}'.format(config['model_name_or_path']))
        if "aleatoric_method" in config.keys():
            if 'distilbert' in config['model_name_or_path']:
                config_list.append('+ue.epistemic.use_tanh\=True')
                config_list.append(f'ue.val_path\=../workdir/run_tasks_for_model_series_method_hp_64/distilbert_{reg}_sn')
            elif 'bert' in config['model_name_or_path']:
                config_list.append(f'ue.val_path\=../workdir/run_tasks_for_model_series_method_hp_64/bert_{reg}_sn')
    else:
        if "aleatoric_method" in config.keys():
            config_list.append(f'ue.val_path\=../workdir/run_tasks_for_model_series_method_hp_64/electra_{reg}_sn')

    config_list.append('data.validation_subsample\=0.0')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                script = conf['script']
                new_task += f' script={script}'
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']

                if 'model_name_or_path' in conf.keys():
                    model = conf['model_name_or_path']
                else:
                    model = 'google/electra-base-discriminator'     
                model_name = model.split('/')[-1].split('-')[0]

                args = ' '.join(generate_func[i](conf))

                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                coverage = ""
                if "coverage" in conf.keys():
                    coverage = '_{}'.format(conf['coverage'])

                use_tanh = '_tanh' if conf['use_tanh']=='True' else ''
                base_path=f' output_dir=../workdir/run_tasks_for_model_series_64{use_tanh}_new/{model_name}_{reg}_{use_spectralnorm}{coverage}/{task_name}'

                if conf['ue'] == "hybrid":
                    ue_type = conf['epistemic_ue']
                    aleatoric_method = conf['aleatoric_method']
                    base_path+= f'/hybrid_{ue_type}_{aleatoric_method}'
                else:
                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                
                new_task+=base_path

                model_series_dir=f'/home/jovyan/projects/uncertainty-estimation/workdir/run_train_models/{model_name}_{reg}_{use_spectralnorm}_opt{coverage}/{task_name}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [24]:
tasks = ['paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5', 'amazon', '20newsgroups']

ue_configs = {
    'ue': ['hybrid'],
    'epistemic_ue': [ 'mahalanobis', 'ddu', 'rde'],
    'aleatoric_method': ['sr', 'entropy'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'use_spectralnorm': ['True'],
    'return_features': ['False'],
    'use_tanh': ['False']
}
    
cuda_devices = '[0,1,2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_electra_toxic_huq.sh')

In [25]:
tasks = ['amazon', '20newsgroups', 'paradetox', 'toxigen', 'implicithate', 'jigsaw_toxic', 'twitter_hso', 'sst5']

ue_configs = {
    'ue': ['hybrid'],
    'epistemic_ue': ['mahalanobis', 'ddu', 'rde'],
    'aleatoric_method': ['sr', 'entropy'],
    'reg_type': ['raw'],
    'script': ['run_glue.py'],
    'use_spectralnorm': ['True'],
    'return_features': ['False'],
    'use_tanh': ['False'],
    'model_name_or_path': ['bert-base-uncased']
}
    
cuda_devices = '[0,1]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_bert_toxic_huq.sh')